## Clean Food.com Recipes and Create Recipes Labels

In [ ]:
!pip install openai
!pip install retry
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 980.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from openai import OpenAI
import openai
from retry import retry
import spacy
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import ast
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
updated_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/Updated_GeneratedFeatures_Cook_times_withGPT.csv')
updated_features.head()

,RecipeId,Name,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,Calories,FatContent,...,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,cook_time,region_country,Healthy_Type,Diet_Types
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,[['https://img.sndimg.com/food/image/upload/w_...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",170.9,2.5,...,37.1,3.6,30.2,3.2,4.0,1-) Toss 2 cups berries with sugar.\n2-) Let s...,< 4 Hours,NaN,"['Low Sodium', 'Healthy', 'Low Cholesterol', '...","Standard, Vegetarian"
1,39,Biryani,Make and share this Biryani .,[['https://img.sndimg.com/food/image/upload/w_...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...",1110.7,58.8,...,84.4,9.0,20.4,63.4,6.0,1-) Soak saffron in warm milk for 5 minutes an...,< 60 Mins,"Asian,Indian","['Low Sugar', 'High Calorie', 'Low Carbohydrat...",Standard
2,40,Best Lemonade,This is from one of my first Good House Keepi...,[['https://img.sndimg.com/food/image/upload/w_...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...",311.1,0.2,...,81.5,0.4,77.2,0.3,4.0,1-) Into a 1 quart Jar with tight fitting lid\...,< 60 Mins,NaN,"['High Sugar', 'Medium Calorie', 'Healthy', 'L...","Standard, Vegan, Vegetarian"
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,[['https://img.sndimg.com/food/image/upload/w_...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",536.1,24.0,...,64.2,17.3,32.1,29.3,2.0,1-) Drain the tofu\n2-) carefully squeezing ou...,< 30 Mins,NaN,"['High Fiber Quality', 'High Calorie', 'Low Ch...","Standard, Vegetarian"
4,42,Cabbage Soup,Make and share this Cabbage Soup .,[['https://img.sndimg.com/food/image/upload/w_...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...",103.6,0.4,...,25.1,4.8,17.7,4.3,4.0,1-) Mix everything together and bring to a boi...,< 60 Mins,NaN,"['Healthy', 'Low Cholesterol', 'Low Carbohydra...","Standard, Vegan, Vegetarian"


In [ ]:
print(updated_features.loc[0]['Healthy_Type'])

['Low Sodium', 'Healthy', 'Low Cholesterol', 'Low Carbohydrate', 'Low Protein', 'Low Fat', 'Medium Sugar']


In [ ]:
recipes_to_process = updated_features[['RecipeId', 'Name', 'Keywords']].copy()
recipes_to_process.head()

,RecipeId,Name,Keywords
0,38,Low-Fat Berry Blue Frozen Dessert,"Dessert, Low Protein, Low Cholesterol, Healthy..."
1,39,Biryani,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A..."
2,40,Best Lemonade,"Low Protein, Low Cholesterol, Healthy, Summer,..."
3,41,Carina's Tofu-Vegetable Kebabs,"Beans, Vegetable, Low Cholesterol, Weeknight, ..."
4,42,Cabbage Soup,"Low Protein, Vegan, Low Cholesterol, Healthy, ..."


In [ ]:
client = OpenAI(api_key = 'apikey')

### Analysing based on keywords

In [ ]:
chunk_size = 5000
result_list = []
all_tags = set()

total_chunks = (recipes_to_process.shape[0] + chunk_size - 1) // chunk_size

for start in tqdm(range(0, recipes_to_process.shape[0], chunk_size), total=total_chunks, desc="Processing Chunks"):
    chunk = recipes_to_process.iloc[start:start+chunk_size].copy()

    # Normalize and explode the tags
    chunk.loc[:, 'Keywords'] = chunk['Keywords'].str.split(',')
    exploded = chunk.explode('Keywords')
    exploded.loc[:, 'Keywords'] = exploded['Keywords'].str.strip()

    # Create dummy variables
    dummies = pd.get_dummies(exploded['Keywords'], prefix='keyword')
    exploded = exploded.join(dummies).drop('Keywords', axis=1)

    all_tags.update(dummies.columns)

    # Aggregate data
    aggregated_chunk = exploded.groupby(['RecipeId', 'Name'], as_index=False).sum()
    dummy_columns = dummies.columns
    aggregated_chunk[dummy_columns] = aggregated_chunk[dummy_columns].clip(upper=1)

    result_list.append(aggregated_chunk)

final_tags = sorted(all_tags)

for i in range(len(result_list)):
    missing_tags = set(final_tags) - set(result_list[i].columns)
    if missing_tags:
        missing_cols_df = pd.DataFrame(0, index=result_list[i].index, columns=sorted(missing_tags))
        result_list[i] = pd.concat([result_list[i], missing_cols_df], axis=1)
    result_list[i] = result_list[i][['RecipeId', 'Name'] + final_tags]

# Concatenate all processed chunks back together
id_tags = pd.concat(result_list)

Processing Chunks: 100%|██████████| 18/18 [00:11<00:00,  1.51it/s]


In [ ]:
id_tags.fillna(0, inplace=True)
id_tags.head()

,RecipeId,Name,keyword_< 15 Mins,keyword_< 30 Mins,keyword_< 4 Hours,keyword_< 60 Mins,keyword_African,keyword_Apple,keyword_Artichoke,keyword_Asian,...,keyword_Welsh,keyword_White Rice,keyword_Whitefish,keyword_Whole Chicken,keyword_Whole Duck,keyword_Whole Turkey,keyword_Wild Game,keyword_Winter,keyword_Yam/Sweet Potato,keyword_Yeast Breads
0,38,Low-Fat Berry Blue Frozen Dessert,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39,Biryani,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,40,Best Lemonade,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41,Carina's Tofu-Vegetable Kebabs,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,42,Cabbage Soup,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
id_tags.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/id_tags.csv', index=False)

In [ ]:
tag_columns = [col for col in id_tags.columns if col.startswith('keyword_')]
tags_only = id_tags[tag_columns].copy()
tag_frequencies = tags_only.sum().sort_values(ascending=False)
tag_frequencies.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/tag_frequencies.csv')

In [ ]:
# categorize tags
tag_categories = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/tag_frequencies.csv')
tag_categories.head()

In [ ]:
tag_categories.columns = ['keyword', 'frequency']

In [ ]:
def clean_tag_name(tag):
    cleaned_tag = tag.replace('keyword_', '').replace('[', '').replace(']', '').replace("'", "").strip().lower()
    return cleaned_tag

In [ ]:
tag_categories['clean_keyword'] = tag_categories['keyword'].apply(clean_tag_name)
tag_categories.head()

,keyword,frequency,clean_keyword
0,keyword_Easy,45131,easy
1,keyword_< 60 Mins,30765,< 60 mins
2,keyword_< 30 Mins,22644,< 30 mins
3,keyword_Meat,22412,meat
4,keyword_< 4 Hours,20223,< 4 hours


In [ ]:
with open('clean_tag.txt', 'w') as file:
    for i in tag_categories['clean_keyword']:
        file.write(str(i) + '\n')

In [ ]:
def group_tags(row, categories):
    keyword = row['clean_keyword']
    prompt = f"Given the keyword '{keyword}', please categorize it under one of these categories: {', '.join(categories)}. "
    prompt += "For example:"
    prompt += " - 'American', 'Italian', 'Mexican' should be categorized as 'region'."
    prompt += " - '< 30 mins', '< 4 hours' should be categorized as 'cooking_time'."
    prompt += " - 'Vegan', 'Vegetarian' should be categorized as 'diet_type'."
    prompt += " - 'Gluten', 'Dairy' should be categorized as 'allergy'."
    prompt += "If the tag does not fit any of these categories, categorize it as 'uncategorized'."
    prompt += "Please provide only the category name for the keyword '{keyword}'."

    messages = [
        {
            "role": "system",
            "content": "You are a knowledgeable assistant trained to categorize culinary tags into specific groups. Provide a single category based on the description."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]

    # Send the prompt to the GPT API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    # Extract and return the response
    text = response.choices[0].message.content.strip()
    return text

In [ ]:
@retry(exceptions=Exception, delay=1, backoff=2, max_delay=120)
def group_tags_by_categories(keyword, categories):
    row = {'clean_keyword': keyword}
    category_tag = group_tags(row, categories)
    return category_tag

In [ ]:
categories = ['region', 'cooking_time', 'dish_type', 'diet_type', 'cooking_method', 'occasion', 'health_focus']

In [ ]:
sample_tags = ['preparation', 'course','time-to-make','main-ingredient','dietary']
sample_category = []
for i in sample_tags:
    cate = [group_tags_by_categories(i, categories)]
    sample_category.append(cate)
#sample_category = [group_tags_by_categories(sample_tags, categories)]
print(sample_category)

[['cooking_method'], ['uncategorized'], ['cooking_time'], ['uncategorized'], ['diet_type']]


In [ ]:
tqdm.pandas(desc='categorizing tags')
tag_categories['category'] = tag_categories.progress_apply(group_tags_by_categories, categories=categories, axis=1)

categorizing tags: 100%|██████████| 285/285 [01:59<00:00,  2.38it/s]


In [ ]:
tag_categories.head()

,keyword,frequency,clean_keyword,category
0,keyword_Easy,45131,easy,cooking_difficulty
1,keyword_< 60 Mins,30765,< 60 mins,cooking_time
2,keyword_< 30 Mins,22644,< 30 mins,cooking_time
3,keyword_Meat,22412,meat,dish_type
4,keyword_< 4 Hours,20223,< 4 hours,cooking_time


In [ ]:
# number of categories distribution
category_counts = tag_categories['category'].value_counts()
print('number of tags per category:\n', category_counts)

number of tags per category:
 category
dish_type                                                              93
region                                                                 77
occasion                                                               24
uncategorized                                                          24
health_focus                                                           20
cooking_method                                                         17
diet_type                                                              15
cooking_time                                                            4
category: dish_type                                                     2
category: uncategorized                                                 1
This keyword should be categorized as 'dish_type'.                      1
The keyword 'lemon' should be categorized as 'ingredient'.              1
The keyword 'beverages' should be categorized as 'dish_type'.           1

In [ ]:
cuisine_rows = tag_categories[tag_categories['category'] == 'occasion']
print(cuisine_rows['clean_keyword'].unique())

['for large groups' 'weeknight' 'brunch' 'summer' 'potluck' 'winter'
 'christmas' 'toddler friendly' 'thanksgiving' 'spring' 'camping'
 'st. patricks day' 'halloween' 'birthday' 'hanukkah' 'ramadan'
 'reynolds wrap contest' 'labor day' 'memorial day' 'chinese new year'
 'college food' 'halloween cocktail' 'punch beverage'
 'served hot new years']


In [ ]:
dish_rows = tag_categories[tag_categories['category'] == 'dish_type']
print(dish_rows['clean_keyword'].unique())

['meat' 'vegetable' 'poultry' 'chicken' 'dessert' 'cheese'
 'cookie & brownie' 'breads' 'pork' 'beans' 'lunch/snacks' 'potato' 'rice'
 'apple' 'breakfast' 'chicken breast' 'strawberry' 'one dish meal'
 'pineapple' 'ham' 'chicken thigh & leg' 'black beans' 'spaghetti'
 'quick breads' 'pears' 'lamb/sheep' 'steak' 'tuna' 'cauliflower' 'penne'
 'yam/sweet potato' 'crab' 'wild game' 'roast beef' 'whole chicken'
 'mango' 'stew' 'pasta shells' 'pumpkin' 'duck' 'oysters' 'yeast breads'
 'tilapia' 'turkey breasts' 'veal' 'frozen desserts' 'halibut' 'shakes'
 'savory pies' 'mussels' 'beef liver' 'lobster' 'pie' 'collard greens'
 'crawfish' 'manicotti' 'artichoke' 'cheesecake' 'chocolate chip cookies'
 'catfish' 'squid' 'ice cream' 'bar cookie' 'rabbit' 'spreads' 'whitefish'
 'mahi mahi' 'chicken livers' 'chowders' 'trout' 'duck breasts' 'bass'
 'elk' 'salad dressings' 'meatballs' 'meatloaf' 'orange roughy'
 'whole duck' 'whole turkey' 'pheasant' 'oatmeal' 'octopus' 'pasta elbow'
 'perch' 'mashed

In [ ]:
taste_mood_rows = tag_categories[tag_categories['category'].str.len()>=15]
print(taste_mood_rows['category'].unique())
print(taste_mood_rows['clean_keyword'].unique())

['cooking_difficulty'
 "The keyword 'beginner cook' should be categorized as 'skill_level'."
 'category: dish_type'
 "The keyword 'savory' should be categorized as 'taste_profile'."
 'category: uncategorized'
 "The keyword 'beverages' should be categorized as 'dish_type'."
 "The keyword 'lemon' should be categorized as 'ingredient'."
 "This keyword should be categorized as 'dish_type'."
 "The keyword 'candy' should be categorized as 'dish_type'."]
['easy' 'beginner cook' 'fruit' 'savory' 'onions' 'beverages' 'lemon'
 'white rice' 'sauces' 'candy']


In [ ]:
tag_categories['category'] = tag_categories['category'].replace({
    "category: dish_type": 'dish_type',
    "category: uncategorized": 'uncategorized',
    "This keyword should be categorized as 'dish_type'.": 'dish_type',
    "The keyword 'lemon' should be categorized as 'ingredient'.": 'uncategorized',
    "The keyword 'beverages' should be categorized as 'dish_type'.": 'dish_type',
    "cooking_difficulty": 'uncategorized',
    "grains": 'uncategorized',
    "The keyword 'savory' should be categorized as 'taste_profile'.": 'uncategorized',
    "The keyword 'beginner cook' should be categorized as 'skill_level'.": 'uncategorized',
    "The keyword 'candy' should be categorized as 'dish_type'.": 'uncategorized'
})

In [ ]:
category_counts_updated = tag_categories['category'].value_counts()
print('number of tags per category:\n', category_counts_updated)

number of tags per category:
 category
dish_type         97
region            77
uncategorized     31
occasion          24
health_focus      20
cooking_method    17
diet_type         15
cooking_time       4
Name: count, dtype: int64


In [ ]:
tag_categories.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/tag_with_categories.csv', index=False)

### Normalize Existing tags

In [ ]:
tag_with_categories = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/tag_with_categories.csv')
tag_with_categories.head()

In [ ]:
tag_to_category_map = dict(zip(tag_with_categories['clean_keyword'], tag_with_categories['category']))
# recipes_categories = recipes_tags.copy()
# recipes_categories.head()

In [ ]:
# Removing tags belonging to 'ingredient_focus' and 'uncategorized'
tag_to_category_map = {tag: cat for tag, cat in tag_to_category_map.items() if cat not in ['uncategorized']}
tag_with_categories = tag_with_categories[~tag_with_categories['category'].isin(['uncategorized'])]

In [ ]:
tag_with_categories['category'].value_counts()

category
dish_type         97
region            77
occasion          24
health_focus      20
cooking_method    17
diet_type         15
cooking_time       4
Name: count, dtype: int64

In [ ]:
tag_with_categories['clean_keyword'] = tag_with_categories['clean_keyword'].astype(str)
category_groups = tag_with_categories.groupby('category')['clean_keyword'].apply(list).to_dict()

nlp = spacy.load("en_core_web_lg")

def cluster_tags_by_category(tag_with_categories):
    grouped = tag_with_categories.groupby('category')
    tag_with_categories['cluster'] = None

    for name, group in grouped:
        tags = group['clean_keyword'].tolist()
        embeddings = np.array([nlp(tag).vector for tag in tags])
        clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.85, affinity='euclidean', linkage='ward')
        labels = clustering.fit_predict(embeddings)
        tag_with_categories.loc[group.index, 'cluster'] = labels
    return tag_with_categories

tag_with_categories = cluster_tags_by_category(tag_with_categories)
print(tag_with_categories)

<ipython-input-45-bcec75ed0ceb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tag_with_categories['clean_keyword'] = tag_with_categories['clean_keyword'].astype(str)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed 

                          keyword  frequency         clean_keyword  \
1               keyword_< 60 Mins      30765             < 60 mins   
2               keyword_< 30 Mins      22644             < 30 mins   
3                    keyword_Meat      22412                  meat   
4               keyword_< 4 Hours      20223             < 4 hours   
5               keyword_Vegetable      16876             vegetable   
..                            ...        ...                   ...   
280                 keyword_Gumbo          1                 gumbo   
281  keyword_Served Hot New Years          1  served hot new years   
282        keyword_Eggs Breakfast          1        eggs breakfast   
283         keyword_Desserts Easy          1         desserts easy   
284        keyword_Beef Crock Pot          1        beef crock pot   

           category cluster  
1      cooking_time       2  
2      cooking_time       3  
3         dish_type      79  
4      cooking_time       1  
5        

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [ ]:
# find centroid tags per category
def indentify_centroids_by_category(tag_with_categories):
    centroids_info = {}
    grouped = tag_with_categories.groupby(['category', 'cluster'])

    for (category, cluster), group in grouped:
        tags = group['clean_keyword'].tolist()
        embeddings = np.array([nlp(tag).vector for tag in tags])
        centroid = np.mean(embeddings, axis=0)
        closest_index = np.argmin(np.linalg.norm(embeddings - centroid, axis=1))
        centroid_tag = tags[closest_index]
        centroids_info[(category, cluster)] = centroid_tag

    return centroids_info

centroids = indentify_centroids_by_category(tag_with_categories)
tag_with_categories['centroid_keyword'] = tag_with_categories.apply(lambda row: centroids[(row['category'], row['cluster'])], axis=1)
print(tag_with_categories)

                          keyword  frequency         clean_keyword  \
1               keyword_< 60 Mins      30765             < 60 mins   
2               keyword_< 30 Mins      22644             < 30 mins   
3                    keyword_Meat      22412                  meat   
4               keyword_< 4 Hours      20223             < 4 hours   
5               keyword_Vegetable      16876             vegetable   
..                            ...        ...                   ...   
280                 keyword_Gumbo          1                 gumbo   
281  keyword_Served Hot New Years          1  served hot new years   
282        keyword_Eggs Breakfast          1        eggs breakfast   
283         keyword_Desserts Easy          1         desserts easy   
284        keyword_Beef Crock Pot          1        beef crock pot   

           category cluster      centroid_keyword  
1      cooking_time       2             < 60 mins  
2      cooking_time       3             < 30 mins  
3  

In [ ]:
tag_with_categories.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/tag_with_cluster.csv', index=False)

In [ ]:
# load the renamed  tags
renamed_tags = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/renamed_tags.csv', sep=';')
renamed_tags.head()

In [ ]:
renamed_tags['category'].unique()

array(['cooking_time', 'dish_type', 'health_focus', 'diet_type',
       'cooking_method', 'uncategorised', 'region', 'occasion'],
      dtype=object)

### Map the Categories with Recipe data

In [ ]:
tag_with_categories = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/tag_with_categories.csv')
tag_with_categories.head()

,keyword,frequency,clean_keyword,category
0,keyword_Easy,45131,easy,uncategorized
1,keyword_< 60 Mins,30765,< 60 mins,cooking_time
2,keyword_< 30 Mins,22644,< 30 mins,cooking_time
3,keyword_Meat,22412,meat,dish_type
4,keyword_< 4 Hours,20223,< 4 hours,cooking_time


In [ ]:
recipes_categories = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/id_tags.csv')
recipes_categories.head()

,RecipeId,Name,keyword_< 15 Mins,keyword_< 30 Mins,keyword_< 4 Hours,keyword_< 60 Mins,keyword_African,keyword_Apple,keyword_Artichoke,keyword_Asian,...,keyword_Welsh,keyword_White Rice,keyword_Whitefish,keyword_Whole Chicken,keyword_Whole Duck,keyword_Whole Turkey,keyword_Wild Game,keyword_Winter,keyword_Yam/Sweet Potato,keyword_Yeast Breads
0,38,Low-Fat Berry Blue Frozen Dessert,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39,Biryani,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,40,Best Lemonade,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41,Carina's Tofu-Vegetable Kebabs,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,42,Cabbage Soup,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
# load the renamed tags
renamed_tags = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/renamed_tags.csv', sep=';')
renamed_tags.head()

,keyword,frequency,clean_keyword,category,cluster,centroid_keyword
0,keyword_< 60 Mins,30765,< 60 mins,cooking_time,2,< 60 mins
1,keyword_< 30 Mins,22644,< 30 mins,cooking_time,3,< 30 mins
2,keyword_< 4 Hours,20223,< 4 hours,cooking_time,1,< 4 hours
3,keyword_< 15 Mins,8688,< 15 mins,cooking_time,0,< 15 mins
4,keyword_Dessert,6105,dessert,dish_type,89,dessert


In [ ]:
# map the tags with centroid tags
tag_to_centroid = dict(zip(renamed_tags['clean_keyword'], renamed_tags['centroid_keyword']))
tag_to_category = dict(zip(renamed_tags['clean_keyword'], renamed_tags['category']))

In [ ]:
for tag_col in recipes_categories.columns:
    if tag_col.startswith('keyword_'):
        tag = tag_col.replace('keyword_', '').replace('[', '').replace(']', '').replace("'", "").strip().lower()
        centroid_tag = tag_to_centroid.get(tag, tag)
        category = tag_to_category.get(tag)

        if category:
            if category not in recipes_categories:
                recipes_categories[category] = np.nan
            recipes_categories.loc[recipes_categories[tag_col] >= 1, category] = (
                recipes_categories.loc[recipes_categories[tag_col] >= 1, category].astype(str) + ', ' + centroid_tag
            ).str.strip(', ').str.split(', ').apply(set).str.join(', ')

recipes_categories.drop(columns=[col for col in recipes_categories.columns if col.startswith('keyword_')], inplace=True)
recipes_categories.head()

,RecipeId,Name,cooking_time,region,uncategorised,health_focus,cooking_method,dish_type,occasion,diet_type
0,38,Low-Fat Berry Blue Frozen Dessert,NaN,NaN,NaN,"nan, low protein, low cholesterol, healthy",NaN,"nan, dessert","weeknight, nan, summer","nan, free of..."
1,39,Biryani,NaN,"indian, nan, asian","poultry, chicken thigh & leg, nan, chicken, meat",NaN,"nan, stove top",NaN,"weeknight, nan",NaN
2,40,Best Lemonade,"< 60 mins, nan",NaN,NaN,"nan, low protein, low cholesterol, healthy",NaN,NaN,"nan, summer",NaN
3,41,Carina's Tofu-Vegetable Kebabs,NaN,NaN,"nan, vegetable, beans","nan, low cholesterol","nan, broil/grill, oven",NaN,"weeknight, nan",NaN
4,42,Cabbage Soup,"< 60 mins, nan",NaN,NaN,"nan, low protein, low cholesterol, healthy",NaN,NaN,"winter, nan","nan, vegan"


In [ ]:
def clean_cell(cell):
    if isinstance(cell, str):
        items = cell.split(',')
        cleaned_items = [item.strip() for item in items if item.strip().lower() != 'nan']
        return ', '.join(cleaned_items)
    else:
        return ''

recipes_categories.drop(columns=['uncategorised', 'cooking_time', 'region', 'health_focus', 'diet_type'], inplace=True)

columns_to_clean = ['occasion', 'cooking_method', 'dish_type']
for column in columns_to_clean:
    if column in recipes_categories.columns:
        recipes_categories[column] = recipes_categories[column].apply(clean_cell)

print(recipes_categories.dtypes)

RecipeId           int64
Name              object
cooking_method    object
dish_type         object
occasion          object
dtype: object


In [ ]:
recipes_categories.replace('', np.nan, inplace=True)
recipes_categories.head()

,RecipeId,Name,cooking_method,dish_type,occasion
0,38,Low-Fat Berry Blue Frozen Dessert,NaN,dessert,"weeknight, summer"
1,39,Biryani,stove top,NaN,weeknight
2,40,Best Lemonade,NaN,NaN,summer
3,41,Carina's Tofu-Vegetable Kebabs,"broil/grill, oven",NaN,weeknight
4,42,Cabbage Soup,NaN,NaN,winter


In [ ]:
recipes_categories.notnull().sum()

RecipeId          88731
Name              88731
cooking_method    22434
dish_type         16082
occasion          17824
dtype: int64

In [ ]:
recipes_categories.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipes_categories_based_on_keyword.csv', index=False)

### Add more features with gpt

In [ ]:
recipes_total = updated_features.copy()

In [ ]:
recipes_total.columns

Index(['RecipeId', 'Name', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeServings', 'RecipeInstructions', 'cook_time',
       'region_country', 'Healthy_Type', 'Diet_Types'],
      dtype='object')

In [ ]:
recipes_categories = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipes_categories_based_on_keyword.csv')
recipes_categories.head()

,RecipeId,Name,dish_type,cooking_method,occasion
0,38,Low-Fat Berry Blue Frozen Dessert,dessert,NaN,"weeknight, summer"
1,39,Biryani,"chicken, chicken thigh & leg, poultry",stove top,weeknight
2,40,Best Lemonade,NaN,NaN,summer
3,41,Carina's Tofu-Vegetable Kebabs,"vegetable, beans","oven, broil/grill",weeknight
4,42,Cabbage Soup,NaN,NaN,winter


In [ ]:
text_to_use = recipes_total[['RecipeId', 'Name', 'Description','Keywords', 'RecipeIngredientParts', 'RecipeInstructions']].copy()
text_to_use.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"Dessert, Low Protein, Low Cholesterol, Healthy...","blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...
1,39,Biryani,Make and share this Biryani .,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"Low Protein, Low Cholesterol, Healthy, Summer,...","sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...


In [ ]:
recipes_with_original_features = text_to_use.merge(recipes_categories[['RecipeId', 'occasion']], on='RecipeId', how='left')
recipes_with_gpt_features = recipes_with_original_features.copy()
recipes_with_gpt_features['occasion_gpt'] = np.nan
recipes_with_gpt_features.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,occasion,occasion_gpt
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"Dessert, Low Protein, Low Cholesterol, Healthy...","blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,"weeknight, summer",NaN
1,39,Biryani,Make and share this Biryani .,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,weeknight,NaN
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"Low Protein, Low Cholesterol, Healthy, Summer,...","sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,summer,NaN
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,weeknight,NaN
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,winter,NaN


In [ ]:
def occasion_prompt(row):
    existing_occasion = row['occasion'] if not pd.isna(row['occasion']) and 'null' not in str(row['occasion']).lower() else "unknown"
    prompt = f"""
    Given the recipe name '{row['Name']}', ingredients: {row['RecipeIngredientParts']}', and a brief description: {row['Description'][:150]}. The current occasion is {existing_occasion}. What is the likely dish occasion? Respond only with the occasion, using the format 'occasion'. If the dish is common in multiple occasions, list at most top 3 relevant occasions. For example, if the dish is for Christmas, answer 'Christmas'. If the dish is for winter and birthday, answer 'winter, birthday'. If the dish is not related to any specific occasion, answer 'general'.
    """
    return prompt

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def classify_occasion(prompt, max_retries=5):
    messages = [
        {
            "role": "system",
            "content": "You are a knowledgeable assistant trained to label the recipe for different occasions."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=0,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                max_tokens=30
            )
            text = response.choices[0].message.content.strip()
            return text
        except Exception as e:
            wait_time = 2 ** attempt
            logger.warning(f"Get problem {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            continue
    return None


def needs_update(value):
    return pd.isna(value) or 'null' in str(value).lower()

In [ ]:
# Process each row conditionally based on needs_update
updated_row_num = 0
for index, row in tqdm(recipes_with_gpt_features.iterrows(), total=recipes_with_gpt_features.shape[0], desc='updating occasion for recipes'):
    if needs_update(row['occasion']):
        prompt = occasion_prompt(row)
        occasion = classify_occasion(prompt)
        recipes_with_gpt_features.at[index, 'occasion_gpt'] = occasion
        updated_row_num += 1
print(f"Updated {updated_row_num} row with new occasion data.")

updating occasion for recipes: 100%|██████████| 88731/88731 [6:36:04<00:00,  3.73it/s]

Updated 58796 row with new occasion data.


In [ ]:
recipes_with_gpt_features.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,occasion,occasion_gpt
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"Dessert, Low Protein, Low Cholesterol, Healthy...","blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,"summer, weeknight",NaN
1,39,Biryani,Make and share this Biryani .,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,weeknight,NaN
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"Low Protein, Low Cholesterol, Healthy, Summer,...","sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,summer,NaN
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,weeknight,NaN
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,winter,NaN


In [ ]:
recipes_with_gpt_features.isnull().sum()

RecipeId                     0
Name                         0
Description                  0
Keywords                     0
RecipeIngredientParts        0
RecipeInstructions           0
occasion                 58796
occasion_gpt             29935
dtype: int64

In [ ]:
recipes_with_gpt_features.shape

(88731, 8)

In [ ]:
recipes_with_gpt_features.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipes_with_gpt_features.csv', index=False)

In [ ]:
data = {
    'RecipeId': [1, 2],
    'Name': ['Christmas Cookies', 'Punch'],
    'Description': [
        'Delicious cookies perfect for Christmas.',
        'A spooky punch for parties.'
    ],
    'Keywords': ['Christmas, cookies', 'autumn, punch, party'],
    'RecipeIngredientParts': ['flour, sugar, butter', 'fruit punch, soda, ice'],
    'RecipeInstructions': ['Mix ingredients and bake.', 'Mix all ingredients and serve chilled.'],
    'dish_type': [np.nan, np.nan],
    'occasion': [np.nan, np.nan]
}
test_df = pd.DataFrame(data)

In [ ]:
# Process each row conditionally based on needs_update
test_df['occasion_gpt'] = np.nan
for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0], desc='updating dish type and occasion for recipes'):
    if needs_update(row['occasion']):
        prompt = occasion_prompt(row)
        occasion = classify_occasion(prompt)
        test_df.at[index, 'occasion_gpt'] = occasion

updating dish type and occasion for recipes: 100%|██████████| 2/2 [00:00<00:00,  2.81it/s]


In [ ]:
test_df

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,dish_type,occasion,occasion_gpt
0,1,Christmas Cookies,Delicious cookies perfect for Christmas.,"Christmas, cookies","flour, sugar, butter",Mix ingredients and bake.,NaN,NaN,Christmas
1,2,Punch,A spooky punch for parties.,"autumn, punch, party","fruit punch, soda, ice",Mix all ingredients and serve chilled.,NaN,NaN,Halloween


In [ ]:
recipes_dishtype = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/corrected_meal_data.csv', index_col='Unnamed: 0')
recipes_dishtype.head()

,RecipeId,Name,Description,Images,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,...,RecipeInstructions,cook_time,Diet_Types,ScrapedIngredients,Healthy_Type,RecipeIngredientQuantities,RecipeIngredientParts,RegionCountry,URL,meal_type
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,[['https://img.sndimg.com/food/image/upload/w_...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...",170.9,2.5,1.3,8.0,...,1-) Toss 2 cups berries with sugar.\n2-) Let s...,more than 12 Hours,"Standard, Vegetarian","['4 cupsblueberries, fresh or frozen', '1⁄4 cu...","['Low Protein', 'Low Fat', 'Medium Fiber', 'Me...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",Global Food,https://www.food.com/recipe/low-fat-berry-blue...,Dessert
1,39,Biryani,Make and share this Biryani .,[['https://img.sndimg.com/food/image/upload/w_...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...",1110.7,58.8,16.6,372.8,...,1-) Soak saffron in warm milk for 5 minutes an...,less than 6 Hours,Standard,"['1 tablespoonsaffron', '4 teaspoonsmilk, warm...","['High Calorie', 'High Cholesterol', 'High Pro...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...","Asian,Indian",https://www.food.com/recipe/biryani-39,Dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,[['https://img.sndimg.com/food/image/upload/w_...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...",311.1,0.2,0.0,0.0,...,1-) Into a 1 quart Jar with tight fitting lid\...,less than 60 Mins,"Standard, Vegan, Vegetarian","['11⁄2 cupssugar', '1 tablespoonlemons, rind o...","['Low Protein', 'Medium Calorie', 'Low Fat', '...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...","North America, United States.",https://www.food.com/recipe/best-lemonade-40,Drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,[['https://img.sndimg.com/food/image/upload/w_...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...",536.1,24.0,3.8,0.0,...,1-) Drain the tofu\n2-) carefully squeezing ou...,more than 12 Hours,"Standard, Vegetarian","['12 ouncesextra firm tofu, water-packed', '1 ...","['Medium Calorie', 'High Protein', 'High Sodiu...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",Global Food,https://www.food.com/recipe/carina%27s-tofu-ve...,Dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,[['https://img.sndimg.com/food/image/upload/w_...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...",103.6,0.4,0.1,0.0,...,1-) Mix everything together and bring to a boi...,less than 60 Mins,"Standard, Vegan, Vegetarian","['46 ouncesplain tomato juice', '4 cupscabbage...","['Low Carb', 'Low Protein', 'Low Fat', 'High S...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...","North America, United States",https://www.food.com/recipe/cabbage-soup-42,Dinner


In [ ]:
recipes_dishtype['meal_type'].value_counts()

meal_type
Dinner       50191
Dessert      14964
Lunch         7788
Breakfast     6789
Brunch        4554
Snack         3221
Drink         1072
Starter        141
Name: count, dtype: int64

In [ ]:
recipes_with_gpt_features['occasion_updated'] = recipes_with_gpt_features['occasion'].combine_first(recipes_with_gpt_features['occasion_gpt'])
recipes_with_gpt_features.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,occasion,occasion_gpt,occasion_updated
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"Dessert, Low Protein, Low Cholesterol, Healthy...","blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,"summer, weeknight",NaN,"summer, weeknight"
1,39,Biryani,Make and share this Biryani .,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,weeknight,NaN,weeknight
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"Low Protein, Low Cholesterol, Healthy, Summer,...","sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,summer,NaN,summer
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,weeknight,NaN,weeknight
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,winter,NaN,winter


In [ ]:
recipes_0719 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipeGenerated5 (1).csv', index_col='Unnamed: 0')
recipes_0719.head()

,RecipeId,Name,Description,Images,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,...,RecipeServings,RecipeInstructions,cook_time,Diet_Types,ScrapedIngredients,Healthy_Type,RecipeIngredientQuantities,RecipeIngredientParts,RegionCountry,URL
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,[['https://img.sndimg.com/food/image/upload/w_...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...",170.9,2.5,1.3,8.0,...,4.0,1-) Toss 2 cups berries with sugar.\n2-) Let s...,more than 12 Hours,"Standard, Vegetarian","['4 cupsblueberries, fresh or frozen', '1⁄4 cu...","['Low Protein', 'Low Fat', 'Medium Fiber', 'Me...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",Global Food,https://www.food.com/recipe/low-fat-berry-blue...
1,39,Biryani,Make and share this Biryani .,[['https://img.sndimg.com/food/image/upload/w_...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...",1110.7,58.8,16.6,372.8,...,6.0,1-) Soak saffron in warm milk for 5 minutes an...,less than 6 Hours,Standard,"['1 tablespoonsaffron', '4 teaspoonsmilk, warm...","['High Calorie', 'High Cholesterol', 'High Pro...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...","Asian,Indian",https://www.food.com/recipe/biryani-39
2,40,Best Lemonade,This is from one of my first Good House Keepi...,[['https://img.sndimg.com/food/image/upload/w_...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...",311.1,0.2,0.0,0.0,...,4.0,1-) Into a 1 quart Jar with tight fitting lid\...,less than 60 Mins,"Standard, Vegan, Vegetarian","['11⁄2 cupssugar', '1 tablespoonlemons, rind o...","['Low Protein', 'Medium Calorie', 'Low Fat', '...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...","North America, United States.",https://www.food.com/recipe/best-lemonade-40
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,[['https://img.sndimg.com/food/image/upload/w_...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...",536.1,24.0,3.8,0.0,...,2.0,1-) Drain the tofu\n2-) carefully squeezing ou...,more than 12 Hours,"Standard, Vegetarian","['12 ouncesextra firm tofu, water-packed', '1 ...","['Medium Calorie', 'High Protein', 'High Sodiu...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",Global Food,https://www.food.com/recipe/carina%27s-tofu-ve...
4,42,Cabbage Soup,Make and share this Cabbage Soup .,[['https://img.sndimg.com/food/image/upload/w_...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...",103.6,0.4,0.1,0.0,...,4.0,1-) Mix everything together and bring to a boi...,less than 60 Mins,"Standard, Vegan, Vegetarian","['46 ouncesplain tomato juice', '4 cupscabbage...","['Low Carb', 'Low Protein', 'Low Fat', 'High S...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...","North America, United States",https://www.food.com/recipe/cabbage-soup-42


In [ ]:
recipes_0719 = recipes_0719.merge(recipes_with_gpt_features[['RecipeId', 'occasion_updated']], on='RecipeId', how='left').merge(recipes_dishtype[['RecipeId', 'meal_type']], on='RecipeId', how='left')
recipes_0719.head()

,RecipeId,Name,Description,Images,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,...,cook_time,Diet_Types,ScrapedIngredients,Healthy_Type,RecipeIngredientQuantities,RecipeIngredientParts,RegionCountry,URL,occasion_updated,meal_type
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,[['https://img.sndimg.com/food/image/upload/w_...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...",170.9,2.5,1.3,8.0,...,more than 12 Hours,"Standard, Vegetarian","['4 cupsblueberries, fresh or frozen', '1⁄4 cu...","['Low Protein', 'Low Fat', 'Medium Fiber', 'Me...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",Global Food,https://www.food.com/recipe/low-fat-berry-blue...,"summer, weeknight",Dessert
1,39,Biryani,Make and share this Biryani .,[['https://img.sndimg.com/food/image/upload/w_...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...",1110.7,58.8,16.6,372.8,...,less than 6 Hours,Standard,"['1 tablespoonsaffron', '4 teaspoonsmilk, warm...","['High Calorie', 'High Cholesterol', 'High Pro...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...","Asian,Indian",https://www.food.com/recipe/biryani-39,weeknight,Dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,[['https://img.sndimg.com/food/image/upload/w_...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...",311.1,0.2,0.0,0.0,...,less than 60 Mins,"Standard, Vegan, Vegetarian","['11⁄2 cupssugar', '1 tablespoonlemons, rind o...","['Low Protein', 'Medium Calorie', 'Low Fat', '...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...","North America, United States.",https://www.food.com/recipe/best-lemonade-40,summer,Drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,[['https://img.sndimg.com/food/image/upload/w_...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...",536.1,24.0,3.8,0.0,...,more than 12 Hours,"Standard, Vegetarian","['12 ouncesextra firm tofu, water-packed', '1 ...","['Medium Calorie', 'High Protein', 'High Sodiu...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",Global Food,https://www.food.com/recipe/carina%27s-tofu-ve...,weeknight,Dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,[['https://img.sndimg.com/food/image/upload/w_...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...",103.6,0.4,0.1,0.0,...,less than 60 Mins,"Standard, Vegan, Vegetarian","['46 ouncesplain tomato juice', '4 cupscabbage...","['Low Carb', 'Low Protein', 'Low Fat', 'High S...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...","North America, United States",https://www.food.com/recipe/cabbage-soup-42,winter,Dinner


In [ ]:
recipes_0719.isnull().sum()

RecipeId                      0
Name                          0
Description                   0
Images                        0
RecipeCategory                0
Keywords                      0
Calories                      0
FatContent                    0
SaturatedFatContent           0
CholesterolContent            0
SodiumContent                 0
CarbohydrateContent           0
FiberContent                  0
SugarContent                  0
ProteinContent                0
RecipeServings                0
RecipeInstructions            0
cook_time                     0
Diet_Types                    0
ScrapedIngredients            0
Healthy_Type                  0
RecipeIngredientQuantities    0
RecipeIngredientParts         0
RegionCountry                 0
URL                           0
occasion_updated              0
meal_type                     0
dtype: int64

In [ ]:
recipes_0719.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipes_with_all_categories.csv', index=False)

### Updating Meal Type

In [ ]:
recipes_for_dishtype = text_to_use.merge(recipes_categories[['RecipeId', 'dish_type']], on='RecipeId', how='left')
recipes_for_dishtype_gpt = recipes_for_dishtype.copy()
recipes_for_dishtype_gpt.rename(columns={'dish_type': 'meal_type'}, inplace=True)
recipes_for_dishtype_gpt['meal_type_gpt'] = np.nan
recipes_for_dishtype_gpt.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,meal_type,meal_type_gpt
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"Dessert, Low Protein, Low Cholesterol, Healthy...","blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,NaN
1,39,Biryani,Make and share this Biryani .,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,NaN,NaN
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"Low Protein, Low Cholesterol, Healthy, Summer,...","sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,NaN,NaN
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,NaN,NaN
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,NaN,NaN


In [ ]:
def mealtype_prompt(row):
    existing_mealtype = row['meal_type'] if not pd.isna(row['meal_type']) and 'null' not in str(row['meal_type']).lower() else "unknown"
    prompt = f"""
    Given the recipe name '{row['Name']}', ingredients: {row['RecipeIngredientParts']}', and a brief description: {row['Description'][:150]}. The current occasion is {existing_mealtype}. What is the likely meal type? You can only choose the answer from the following 8 types: Dinner, Dessert, Lunch, Breakfast, Brunch, Snack, Drink, Starter. Respond only with the meal type from the provided 8 types, using the format 'meal type'. If the dish can be classified into multiple types, list at most top 3 relevant meal type. For example, if the dish is a cake recipe, answer 'dessert'. If the dish is a pasta, answer 'Lunch, Dinner'. If you don't know which type the dish belong to, answer 'unknown'.
    """
    return prompt

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def classify_mealtype(prompt, max_retries=5):
    messages = [
        {
            "role": "system",
            "content": "You are a knowledgeable assistant trained to label the recipe for different meal type."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=0,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                max_tokens=30
            )
            text = response.choices[0].message.content.strip()
            return text
        except Exception as e:
            wait_time = 2 ** attempt
            logger.warning(f"Get problem {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            continue
    return None


def needs_update(value):
    return pd.isna(value) or 'null' in str(value).lower() or value == 'unknown'

In [ ]:
# Process each row conditionally based on needs_update
updated_row_num = 0
for index, row in tqdm(recipes_for_dishtype_gpt.iterrows(), total=recipes_for_dishtype_gpt.shape[0], desc='updating meal type for recipes'):
    if needs_update(row['meal_type']):
        prompt = mealtype_prompt(row)
        meal_type = classify_mealtype(prompt)
        recipes_for_dishtype_gpt.at[index, 'meal_type_gpt'] = meal_type
        updated_row_num += 1
print(f"Updated {updated_row_num} row with new meal type data.")

updating meal type for recipes: 100%|██████████| 88731/88731 [7:04:45<00:00,  3.48it/s]

Updated 72649 row with new meal type data.


In [ ]:
recipes_for_dishtype_gpt.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,meal_type,meal_type_gpt,mealtype_
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"Dessert, Low Protein, Low Cholesterol, Healthy...","blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,NaN,NaN
1,39,Biryani,Make and share this Biryani .,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,NaN,Meal type.,Meal type.
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"Low Protein, Low Cholesterol, Healthy, Summer,...","sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,NaN,Drink,Drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,NaN,Dinner,Dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,NaN,"Lunch, Dinner","Lunch, Dinner"


In [ ]:
recipes_for_dishtype_gpt.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipes_for_dishtype_gpt.csv', index=False)

In [ ]:
data = {
    'RecipeId': [1, 2],
    'Name': ['Christmas Cookies', 'Punch'],
    'Description': [
        'Delicious cookies perfect for Christmas.',
        'A spooky punch for parties.'
    ],
    'Keywords': ['Christmas, cookies', 'autumn, punch, party'],
    'RecipeIngredientParts': ['flour, sugar, butter', 'fruit punch, soda, ice'],
    'RecipeInstructions': ['Mix ingredients and bake.', 'Mix all ingredients and serve chilled.'],
    'meal_type': [np.nan, np.nan],
    'occasion': [np.nan, np.nan]
}
test_df = pd.DataFrame(data)

In [ ]:
# Process each row conditionally based on needs_update
test_df['mealtype_gpt'] = np.nan
for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0], desc='updating dish type for recipes'):
    if needs_update(row['meal_type']):
        prompt = mealtype_prompt(row)
        mealtype = classify_mealtype(prompt)
        test_df.at[index, 'mealtype_gpt'] = mealtype

updating dish type for recipes: 100%|██████████| 2/2 [00:01<00:00,  1.45it/s]


In [ ]:
test_df

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,meal_type,occasion,mealtype_gpt
0,1,Christmas Cookies,Delicious cookies perfect for Christmas.,"Christmas, cookies","flour, sugar, butter",Mix ingredients and bake.,NaN,NaN,Dessert
1,2,Punch,A spooky punch for parties.,"autumn, punch, party","fruit punch, soda, ice",Mix all ingredients and serve chilled.,NaN,NaN,Drink


In [ ]:
recipes_for_dishtype_gpt.isnull().sum()

RecipeId                     0
Name                         0
Description                  0
Keywords                     0
RecipeIngredientParts        0
RecipeInstructions           0
meal_type                72649
meal_type_gpt            16082
mealtype_                86349
dtype: int64

In [ ]:
recipes_for_dishtype_gpt['meal_type_updated'] = recipes_for_dishtype_gpt['meal_type'].combine_first(recipes_for_dishtype_gpt['meal_type_gpt'])
recipes_for_dishtype_gpt.head()

,RecipeId,Name,Description,Keywords,RecipeIngredientParts,RecipeInstructions,meal_type,meal_type_gpt,mealtype_,meal_type_updated
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"Dessert, Low Protein, Low Cholesterol, Healthy...","blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,NaN,NaN,dessert
1,39,Biryani,Make and share this Biryani .,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...","saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,NaN,Meal type.,Meal type.,Meal type.
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"Low Protein, Low Cholesterol, Healthy, Summer,...","sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,NaN,Drink,Drink,Drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"Beans, Vegetable, Low Cholesterol, Weeknight, ...","extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,NaN,Dinner,Dinner,Dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"Low Protein, Vegan, Low Cholesterol, Healthy, ...","plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,NaN,"Lunch, Dinner","Lunch, Dinner","Lunch, Dinner"


In [ ]:
recipes_for_dishtype_gpt['meal_type_updated'].value_counts()

meal_type_updated
Dinner                                                                                                                                   18263
Dessert                                                                                                                                   9627
Starter                                                                                                                                   9551
Lunch, Dinner                                                                                                                             9051
Meal type.                                                                                                                                7237
                                                                                                                                         ...  
that the likely meal type for the recipe 'Brown Sugar Oatmeal Muffins' would be 'Breakfast'.                                

In [ ]:
meal_type_to_process = recipes_for_dishtype_gpt[['RecipeId', 'Name', 'Description', 'RecipeIngredientParts', 'RecipeInstructions', 'meal_type_updated']].copy()
meal_type_to_process.head()

,RecipeId,Name,Description,RecipeIngredientParts,RecipeInstructions,meal_type_updated
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert
1,39,Biryani,Make and share this Biryani .,"saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,Meal type.
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,Drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,Dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,"Lunch, Dinner"


In [ ]:
meal_type_to_process.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/meal_type_to_process.csv', index=False)

### clean the generated meal type

In [ ]:
meal_type_to_process = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/meal_type_to_process.csv')
meal_type_to_process.head()

,RecipeId,Name,Description,RecipeIngredientParts,RecipeInstructions,meal_type_updated
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert
1,39,Biryani,Make and share this Biryani .,"saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,Meal type.
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,Drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,Dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,"Lunch, Dinner"


In [ ]:
meal_type_to_process['meal_type_updated'] = meal_type_to_process['meal_type_updated'].str.lower().str.strip()
meal_type_to_process['meal_type_updated'].value_counts()

meal_type_updated
dinner                                                                                                                                   18263
dessert                                                                                                                                  14941
starter                                                                                                                                   9552
lunch, dinner                                                                                                                             9051
meal type.                                                                                                                                7239
                                                                                                                                         ...  
that the likely meal type for the recipe 'brown sugar oatmeal muffins' would be 'breakfast'.                                

In [ ]:
mapping = {
    "dinner": "Dinner",
    "dessert": "Dessert",
    "lunch": "Lunch",
    "breakfast": "Breakfast",
    "brunch": "Brunch",
    "snack": "Snack",
    "drink": "Drink",
    "starter": "Starter",
    # Additional mappings for variations and incorrect labels
    "meal type": "Unknown",
    "meal type.": "Unknown",
    "meal type: dinner": "Dinner",
    "meal type: dessert": "Dessert",
    "meal type: lunch": "Lunch",
    "meal type: breakfast": "Breakfast",
    "meal type: snack": "Snack",
    "meal type: drink": "Drink",
    "meal type: starter": "Starter",
    "side dish": "Side Dish",
    "appetizer": "Starter",
    "side": "Side Dish",
    "ersonalized response": "Unknown",
}

meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['meal_type_updated'].replace(mapping)
meal_type_to_process['meal_type_updated'].value_counts()

meal_type_updated
dinner                                                                                                                                   18263
dessert                                                                                                                                  14941
starter                                                                                                                                   9552
lunch, dinner                                                                                                                             9051
meal type.                                                                                                                                7239
                                                                                                                                         ...  
that the likely meal type for the recipe 'brown sugar oatmeal muffins' would be 'breakfast'.                                

In [ ]:
comprehensive_mapping = {
    "dessert": "Dessert",
    "unknown": "Unknown",
    "drink": "Drink",
    "dinner": "Dinner",
    "lunch, dinner": "Dinner, Lunch",
    "starter": "Starter",
    "breakfast": "Breakfast",
    "snack": "Snack",
    "meal type: dinner, lunch, starter": "Dinner, Lunch, Starter",
    "lunch, brunch, snack": "Lunch, Brunch, Snack",
    "brunch": "Brunch",
    "snack, dessert": "Dessert, Snack",
    "dinner, lunch, starter": "Dinner, Lunch, Starter",
    "lunch": "Lunch",
    "lunch, snack, starter": "Lunch, Snack, Starter",
    "lunch, snack": "Lunch, Snack",
    "meal type: breakfast, brunch, snack": "Breakfast, Brunch, Snack",
    "dinner, snack": "Dinner, Snack",
    "lunch, snack, breakfast": "Lunch, Snack, Breakfast",
    "breakfast, snack": "Breakfast, Snack",
    "drink, breakfast": "Drink, Breakfast",
    "lunch, snack, dinner": "Lunch, Snack, Dinner",
    "meal type: dinner, lunch, snack": "Dinner, Lunch, Snack",
    "breakfast, brunch, snack": "Breakfast, Brunch, Snack",
    "dessert, snack, breakfast": "Dessert, Snack, Breakfast",
    "meal type: lunch, dinner, starter": "Lunch, Dinner, Starter",
    "lunch, brunch, dinner": "Lunch, Brunch, Dinner",
    "meal type: breakfast, brunch, lunch": "Breakfast, Brunch, Lunch",
    "breakfast, brunch": "Breakfast, Brunch",
    "snack, lunch": "Snack, Lunch",
    "lunch, starter": "Lunch, Starter",
    "drink, dessert": "Drink, Dessert",
    "snack, breakfast": "Snack, Breakfast",
    "drink, snack": "Drink, Snack",
    "dinner, lunch, snack": "Dinner, Lunch, Snack",
    "lunch, breakfast, brunch": "Lunch, Breakfast, Brunch",
    "starter, lunch, dinner": "Starter, Lunch, Dinner",
    "starter, brunch": "Starter, Brunch",
    "lunch, snack, brunch": "Lunch, Snack, Brunch",
    "breakfast, brunch, lunch": "Breakfast, Brunch, Lunch",
    "dinner, lunch, brunch": "Dinner, Lunch, Brunch",
    "dessert, breakfast": "Dessert, Breakfast",
    "starter, dinner, snack": "Starter, Dinner, Snack",
    "brunch, snack": "Brunch, Snack",
    "side dish": "Starter",
    "starter, snack": "Starter, Snack",
    "meal type: brunch, lunch, breakfast": "Brunch, Lunch, Breakfast",
    "drink, breakfast, snack": "Drink, Breakfast, Snack",
    "breakfast, lunch": "Breakfast, Lunch",
    "dessert, breakfast, snack": "Dessert, Breakfast, Snack",
    "meal type: dinner, lunch, side dish": "Dinner, Lunch, Starter",
    "dessert, snack": "Dessert, Snack",
    "meal type: lunch, snack": "Lunch, Snack",
    "lunch, brunch, breakfast": "Lunch, Brunch, Breakfast",
    "dessert, side, snack": "Dessert, Snack, Starter",
    "meal type: lunch, dinner": "Lunch, Dinner",
    "breakfast, snack, dessert": "Breakfast, Snack, Dessert",
    "meal type: dinner, side dish, lunch": "Dinner, Starter, Lunch",
    "breakfast, snack, brunch": "Breakfast, Snack, Brunch",
    "meal type: snack, lunch, appetizer": "Snack, Lunch, Starter",
    "lunch, dinner, starter": "Lunch, Dinner, Starter",
    "breakfast, lunch, dinner": "Breakfast, Lunch, Dinner",
    "meal type: dinner, lunch, brunch": "Dinner, Lunch, Brunch",
    "breakfast, dessert": "Breakfast, Dessert",
    "lunch, brunch": "Lunch, Brunch",
    "meal type: lunch, dinner, snack": "Lunch, Dinner, Snack",
    "meal type: snack, brunch, starter": "Snack, Brunch, Starter",
    "dinner, brunch, snack": "Dinner, Brunch, Snack",
    "dinner, dessert, snack": "Dinner, Dessert, Snack",
    "breakfast, brunch, dessert": "Breakfast, Brunch, Dessert",
    "lunch, dinner, snack": "Lunch, Dinner, Snack",
    "dinner, side dish, snack": "Dinner, Starter, Snack",
    "drink, breakfast, dessert": "Drink, Breakfast, Dessert",
    "lunch, dinner, brunch": "Lunch, Dinner, Brunch",
    "dinner, snack, starter": "Dinner, Snack, Starter",
    "lunch, starter, snack": "Lunch, Starter, Snack",
    "dinner, side dish, starter": "Dinner, Starter",
    "lunch, brunch, starter": "Lunch, Brunch, Starter",
    "meal type: lunch, dinner, brunch": "Lunch, Dinner, Brunch",
    "meal type: lunch, brunch, dinner": "Lunch, Brunch, Dinner",
    "meal type: lunch, brunch, starter": "Lunch, Brunch, Starter",
    "brunch, dessert": "Brunch, Dessert",
    "brunch, snack, dessert": "Brunch, Snack, Dessert",
    "snack, lunch, breakfast, brunch": "Snack, Lunch, Breakfast, Brunch",
    "dessert, snack, brunch": "Dessert, Snack, Brunch",
    "dessert, breakfast, brunch": "Dessert, Breakfast, Brunch",
    "starter, snack, appetizer": "Starter, Snack, Starter",
    "dinner, starter": "Dinner, Starter",
    "dessert, brunch, snack": "Dessert, Brunch, Snack",
    "brunch, breakfast": "Brunch, Breakfast",
    "dinner, appetizer, snack": "Dinner, Starter, Snack",
    "breakfast, dessert, brunch": "Breakfast, Dessert, Brunch",
    "starter, dinner, lunch": "Starter, Dinner, Lunch",
    "meal type: dinner, side dish, snack": "Dinner, Starter, Snack",
    "brunch, lunch": "Brunch, Lunch",
    "lunch, breakfast": "Lunch, Breakfast",
    "dessert, snack, starter": "Dessert, Snack, Starter",
    "unknown": "Unknown",
    "meal type: dinner, side dish, brunch": "Dinner, Starter, Brunch",
    "the likely meal type is: lunch.": "Lunch",
    "snack, starter": "Snack, Starter",
    "brunch, lunch, snack": "Brunch, Lunch, Snack",
    "meal type: snack, breakfast, brunch": "Snack, Breakfast, Brunch",
    "appetizer, snack": "Starter, Snack",
    "starter, lunch, snack": "Starter, Lunch, Snack",
    "meal type: breakfast, snack, brunch": "Breakfast, Snack, Brunch",
    "drink, brunch": "Drink, Brunch",
    "brunch, breakfast, snack": "Brunch, Breakfast, Snack",
    "starter, snack, dinner": "Starter, Snack, Dinner",
    "meal type: brunch, breakfast, snack": "Brunch, Breakfast, Snack",
    "dinner, lunch, appetizer": "Dinner, Lunch, Starter",
    "breakfast, lunch, brunch": "Breakfast, Lunch, Brunch",
    "starter, appetizer, main dish": "Starter, Starter, Main Dish",
    "drink, dessert, brunch": "Drink, Dessert, Brunch",
    "meal type: dinner, breakfast, lunch": "Dinner, Breakfast, Lunch",
    "meal type: breakfast, dinner, brunch": "Breakfast, Dinner, Brunch",
    "brunch, lunch, dinner": "Brunch, Lunch, Dinner",
    "meal type: breakfast, snack, dessert": "Breakfast, Snack, Dessert",
    "meal type: snack, lunch, dinner": "Snack, Lunch, Dinner",
    "brunch, snack, breakfast": "Brunch, Snack, Breakfast",
    "meal type: breakfast, brunch, dessert": "Breakfast, Brunch, Dessert",
    "meal type: breakfast, brunch": "Breakfast, Brunch",
    "meal type: dinner, snack, party": "Dinner, Snack, Party",
    "dinner, breakfast, lunch": "Dinner, Breakfast, Lunch",
    "dinner, dessert": "Dinner, Dessert",
    "bread is typically consumed during breakfast, lunch, or dinner. so, the likely meal type for squaw bread is 'breakfast, lunch,": "Breakfast, Lunch, Dinner",
    "meal type: lunch, snack, brunch": "Lunch, Snack, Brunch",
    "bread recipes like 'deluxe sourdough bread' are typically associated with the meal type 'breakfast'.": "Breakfast",
    "lunch, breakfast, snack": "Lunch, Breakfast, Snack",
    "main course": "Main Course",
    "dinner, appetizer, lunch": "Dinner, Starter, Lunch",
    "meal type: breakfast, snack, lunch": "breakfast, snack, lunch",
    "bread is typically consumed as a part of a meal rather than as a standalone dish. therefore, the likely meal type for 'rustic country": "Unkown",
    "meal type: dessert, snack": "dessert, snack",
    "meal type.": "Unknown",
    "appetizer, snack, starter": "Starter, Snack",
    "meal type: brunch, snack, starter": "brunch, snack, starter",
    "meal type: dinner, breakfast, brunch": "Dinner, Breakfast, Brunch",
    "- dinner\n- lunch\n- unknown": "Lunch",
    "meal type": "Unknown",
    "meal type: dinner": "dinner",
    "meal type: dessert": "dessert",
    "meal type: starter": "starter",
    "meal type: breakfast": "breakfast",
    "meal type: snack": "snack",
    "appetizer": "starter",
    "meal type: lunch, snack, starter": "lunch, snack, starter",
    "meal type: dinner, snack, starter": "dinner, snack, starter",
    "meal type: breakfast, snack": "breakfast, snack",
    "meal type: dinner, snack": "dinner, snack",
    "Unkown": "Unknown",
    "bread recipes like 'hartness house inn jalapeno bread' are typically served as part of breakfast, lunch, or dinner.": "starter",
    ",,,,,,,,,,,,,,,,,,,,,,,,,,,,,": "Unknown",
    "bread, such as the cream of potato soup bread, is typically categorized as a 'snack' or 'starter'.": "snack, starter",
    "bread is typically consumed during breakfast, lunch, or dinner.": "starter",
    "bread recipes are typically associated with the meal type 'breakfast'.": "starter",
    "bread recipes like 'beth's 100% whole wheat sourdough bread' are typically classified as a **snack** or **starter": "snack, starter",
    "that the likely meal type for the recipe 'brown sugar oatmeal muffins' would be 'breakfast'.": "breakfast",
    "Main Course": "dinner",
    "snack, appetizer, lunch": "snack, starter, lunch"}

meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['meal_type_updated'].replace(comprehensive_mapping)
meal_type_to_process['cleaned_meal_type'].value_counts()

cleaned_meal_type
Dinner                      18263
Dessert                     14941
Unknown                     10308
Starter                      9668
Dinner, Lunch                9051
                            ...  
dessert, starter                1
dinner, dessert, lunch          1
Brunch, Lunch, Snack            1
brunch, lunch, breakfast        1
starter, brunch, snack          1
Name: count, Length: 198, dtype: int64

In [ ]:
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.lower().str.strip().str.replace('meal type: ', '')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('meal type:', '')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('appetizer', 'starter')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('side dish', 'starter')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('side', 'snack')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('dinner, starter, starter', 'starter, snack')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace("st 'starter'", 'starter, snack')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('meal', 'lunch')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('main course', 'dinner')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('dessert dessert', 'dessert')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('unkown', 'unknown')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace(', party,', '')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace('snack, starter, starter', 'starter')

In [ ]:
def replace_if_contains(value, substring, new_value):
    if substring in value:
        return new_value
    return value

meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].apply(lambda x: replace_if_contains(x, 'bread', 'starter'))
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].apply(lambda x: replace_if_contains(x, ', for. claim.', 'unknown'))
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].apply(lambda x: replace_if_contains(x, 'ersonalized response', 'unknown'))
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].apply(lambda x: replace_if_contains(x, 'picnic', 'lunch'))
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].apply(lambda x: replace_if_contains(x, ', party,', 'snack'))
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].apply(lambda x: replace_if_contains(x, 'main,', 'starter'))

In [ ]:
def sort_types(value):
    if pd.isna(value):
        return value
    types_list = value.split(', ')
    types_list.sort()
    return ', '.join(types_list)

In [ ]:
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].apply(sort_types)

In [ ]:
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace("st  'starter'", 'starter')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace("dinner snack", 'dinner, snack')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace("starter snack", 'starter, snack')
meal_type_to_process['cleaned_meal_type'] = meal_type_to_process['cleaned_meal_type'].str.replace("main dish, starter, starter", 'starter')
print(meal_type_to_process['cleaned_meal_type'].value_counts())

cleaned_meal_type
dinner                        19507
dessert                       15027
unknown                       10311
starter                        9703
dinner, lunch                  9130
                              ...  
breakfast, snack, starter         1
dessert, drink, snack             1
brunch, dinner, snack             1
breakfast, brunch, starter        1
dinner, drink                     1
Name: count, Length: 64, dtype: int64


In [ ]:
meal_type_to_process.head()

,RecipeId,Name,Description,RecipeIngredientParts,RecipeInstructions,meal_type_updated,cleaned_meal_type
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,dessert
1,39,Biryani,Make and share this Biryani .,"saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,meal type.,unknown
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,drink,drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,dinner,dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,"lunch, dinner","dinner, lunch"


In [ ]:
value_counts = meal_type_to_process['cleaned_meal_type'].value_counts()
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['meal_type', 'Count']

In [ ]:
value_counts_df.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/meal_type_counts.csv', index=False)

### process the unknown meal type

In [ ]:
def mealtype_prompt_wo_unknown(row):
    existing_mealtype = row['cleaned_meal_type'] if not pd.isna(row['cleaned_meal_type']) and 'null' not in str(row['cleaned_meal_type']).lower() else "unknown"
    prompt = f"""
    Given the recipe name '{row['Name']}', ingredients: {row['RecipeIngredientParts']}', and a brief description: {row['Description'][:150]}. The current occasion is {existing_mealtype}. What is the likely meal type? You can only choose the answer from the following 8 types: Dinner, Dessert, Lunch, Breakfast, Brunch, Snack, Drink, Starter. Respond only with the meal type from the provided 8 types, using the format 'breakfast'. If the dish can be classified into multiple types, list at most top 3 relevant meal type. For example, if the dish is a cake recipe, answer 'dessert'. If the dish is a pasta, answer 'Lunch, Dinner'. Do not answer 'meal type'.
    """
    return prompt

In [ ]:
# Process each row conditionally based on needs_update
updated_row_num = 0
meal_type_to_process['meal_type_latest'] = np.nan
for index, row in tqdm(meal_type_to_process.iterrows(), total=meal_type_to_process.shape[0], desc='updating unknown meal type for recipes'):
    if needs_update(row['cleaned_meal_type']):
        prompt = mealtype_prompt_wo_unknown(row)
        meal_type = classify_mealtype(prompt)
        meal_type_to_process.at[index, 'meal_type_latest'] = meal_type
        updated_row_num += 1
print(f"\nUpdated {updated_row_num} row with previous unknown meal type data.")

updating unknown meal type for recipes: 100%|██████████| 88731/88731 [1:07:23<00:00, 21.94it/s]


Updated 10311 row with previous unknown meal type data.


In [ ]:
meal_type_to_process.head()

,RecipeId,Name,Description,RecipeIngredientParts,RecipeInstructions,meal_type_updated,cleaned_meal_type,meal_type_latest
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,dessert,NaN
1,39,Biryani,Make and share this Biryani .,"saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,meal type.,unknown,Dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,drink,drink,NaN
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,dinner,dinner,NaN
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,"lunch, dinner","dinner, lunch",NaN


In [ ]:
meal_type_to_process.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/meal_type_to_process_latest_undone.csv', index=False)

In [ ]:
updated_meal_type = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/meal_type_to_process_latest_undone.csv')
updated_meal_type.head()

,RecipeId,Name,Description,RecipeIngredientParts,RecipeInstructions,meal_type_updated,cleaned_meal_type,meal_type_latest
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,dessert,NaN
1,39,Biryani,Make and share this Biryani .,"saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,meal type.,unknown,Dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,drink,drink,NaN
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,dinner,dinner,NaN
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,"lunch, dinner","dinner, lunch",NaN


In [ ]:
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.lower().str.strip().str.replace('meal type: ', '')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('meal type:', '')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('meal type.', 'unknown')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('meal type', 'unknown')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('appetizer', 'starter')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('side dish', 'starter')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace(', potluck', '')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace(', vegetarian', '')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('dinner, lunch, lunch', 'dinner, lunch')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('comfort food', 'snack')
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'] .str.replace('main dish', 'dinner')

In [ ]:
updated_meal_type['meal_type_latest'] = updated_meal_type['meal_type_latest'].apply(sort_types)

In [ ]:
updated_meal_type.loc[updated_meal_type['cleaned_meal_type'] == 'unknown', 'cleaned_meal_type'] = updated_meal_type['meal_type_latest']
updated_meal_type.head()

,RecipeId,Name,Description,RecipeIngredientParts,RecipeInstructions,meal_type_updated,cleaned_meal_type,meal_type_latest
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,dessert,NaN
1,39,Biryani,Make and share this Biryani .,"saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,meal type.,dinner,dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,drink,drink,NaN
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,dinner,dinner,NaN
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,"lunch, dinner","dinner, lunch",NaN


In [ ]:
# continute process the unknown meal type
updated_row_num = 0
updated_meal_type['meal_type_2'] = np.nan
for index, row in tqdm(updated_meal_type.iterrows(), total=updated_meal_type.shape[0], desc='updating unknown meal type for recipes'):
    if needs_update(row['cleaned_meal_type']):
        prompt = mealtype_prompt_wo_unknown(row)
        meal_type = classify_mealtype(prompt)
        updated_meal_type.at[index, 'meal_type_2'] = meal_type
        updated_row_num += 1
print(f"\nUpdated {updated_row_num} row with previous unknown meal type data.")

updating unknown meal type for recipes: 100%|██████████| 88731/88731 [28:22<00:00, 52.13it/s]


Updated 4561 row with previous unknown meal type data.


In [ ]:
updated_meal_type.shape

(88731, 9)

In [ ]:
updated_meal_type['meal_type_2'] = updated_meal_type['meal_type_2'].str.lower().str.strip().str.replace('side dish', 'starter')
updated_meal_type['meal_type_2'] = updated_meal_type['meal_type_2'].str.replace('side', 'starter')
updated_meal_type['meal_type_2'] = updated_meal_type['meal_type_2'].str.replace(', celebration', '')
updated_meal_type['meal_type_2'] = updated_meal_type['meal_type_2'].str.replace('potluck', '')
updated_meal_type['meal_type_2'] = updated_meal_type['meal_type_2'].apply(sort_types)

In [ ]:
updated_meal_type.loc[updated_meal_type['cleaned_meal_type'] == 'unknown', 'cleaned_meal_type'] = updated_meal_type['meal_type_2']
updated_meal_type.head()

,RecipeId,Name,Description,RecipeIngredientParts,RecipeInstructions,meal_type_updated,cleaned_meal_type,meal_type_latest,meal_type_2
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,"blueberries, granulated sugar, vanilla yogurt,...",1-) Toss 2 cups berries with sugar.\n2-) Let s...,dessert,dessert,NaN,NaN
1,39,Biryani,Make and share this Biryani .,"saffron, milk, hot green chili peppers, onions...",1-) Soak saffron in warm milk for 5 minutes an...,meal type.,dinner,dinner,NaN
2,40,Best Lemonade,This is from one of my first Good House Keepi...,"sugar, lemons, rind of, lemon, zest of, fresh ...",1-) Into a 1 quart Jar with tight fitting lid\...,drink,drink,NaN,NaN
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,"extra firm tofu, eggplant, zucchini, mushrooms...",1-) Drain the tofu\n2-) carefully squeezing ou...,dinner,dinner,NaN,NaN
4,42,Cabbage Soup,Make and share this Cabbage Soup .,"plain tomato juice, cabbage, onion, carrots, c...",1-) Mix everything together and bring to a boi...,"lunch, dinner","dinner, lunch",NaN,NaN


In [ ]:
updated_meal_type['cleaned_meal_type'] = updated_meal_type['cleaned_meal_type'].apply(sort_types)

In [ ]:
value_counts = updated_meal_type['cleaned_meal_type'].value_counts()
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['meal_type', 'Count']
value_counts_df.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/meal_type_counts.csv', index=False)

### Restructure the whole dataset

In [ ]:
recipes_previous = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipes_with_all_categories.csv')
recipes_previous.head()

,RecipeId,Name,Description,Images,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,...,cook_time,Diet_Types,ScrapedIngredients,Healthy_Type,RecipeIngredientQuantities,RecipeIngredientParts,RegionCountry,URL,occasion_updated,meal_type
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,[['https://img.sndimg.com/food/image/upload/w_...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...",170.9,2.5,1.3,8.0,...,more than 12 Hours,"Standard, Vegetarian","['4 cupsblueberries, fresh or frozen', '1⁄4 cu...","['Low Protein', 'Low Fat', 'Medium Fiber', 'Me...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",Global Food,https://www.food.com/recipe/low-fat-berry-blue...,"summer, weeknight",Dessert
1,39,Biryani,Make and share this Biryani .,[['https://img.sndimg.com/food/image/upload/w_...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...",1110.7,58.8,16.6,372.8,...,less than 6 Hours,Standard,"['1 tablespoonsaffron', '4 teaspoonsmilk, warm...","['High Calorie', 'High Cholesterol', 'High Pro...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...","Asian,Indian",https://www.food.com/recipe/biryani-39,weeknight,Dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,[['https://img.sndimg.com/food/image/upload/w_...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...",311.1,0.2,0.0,0.0,...,less than 60 Mins,"Standard, Vegan, Vegetarian","['11⁄2 cupssugar', '1 tablespoonlemons, rind o...","['Low Protein', 'Medium Calorie', 'Low Fat', '...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...","North America, United States.",https://www.food.com/recipe/best-lemonade-40,summer,Drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,[['https://img.sndimg.com/food/image/upload/w_...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...",536.1,24.0,3.8,0.0,...,more than 12 Hours,"Standard, Vegetarian","['12 ouncesextra firm tofu, water-packed', '1 ...","['Medium Calorie', 'High Protein', 'High Sodiu...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",Global Food,https://www.food.com/recipe/carina%27s-tofu-ve...,weeknight,Dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,[['https://img.sndimg.com/food/image/upload/w_...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...",103.6,0.4,0.1,0.0,...,less than 60 Mins,"Standard, Vegan, Vegetarian","['46 ouncesplain tomato juice', '4 cupscabbage...","['Low Carb', 'Low Protein', 'Low Fat', 'High S...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...","North America, United States",https://www.food.com/recipe/cabbage-soup-42,winter,Dinner


In [ ]:
recipes_previous.shape

(88720, 27)

In [ ]:
recipes_mealtype_nounknown = recipes_previous.merge(updated_meal_type[['RecipeId', 'cleaned_meal_type']], on='RecipeId', how='left')
recipes_mealtype_nounknown.head()

,RecipeId,Name,Description,Images,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,...,Diet_Types,ScrapedIngredients,Healthy_Type,RecipeIngredientQuantities,RecipeIngredientParts,RegionCountry,URL,occasion_updated,meal_type,cleaned_meal_type
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,[['https://img.sndimg.com/food/image/upload/w_...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...",170.9,2.5,1.3,8.0,...,"Standard, Vegetarian","['4 cupsblueberries, fresh or frozen', '1⁄4 cu...","['Low Protein', 'Low Fat', 'Medium Fiber', 'Me...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",Global Food,https://www.food.com/recipe/low-fat-berry-blue...,"summer, weeknight",Dessert,dessert
1,39,Biryani,Make and share this Biryani .,[['https://img.sndimg.com/food/image/upload/w_...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...",1110.7,58.8,16.6,372.8,...,Standard,"['1 tablespoonsaffron', '4 teaspoonsmilk, warm...","['High Calorie', 'High Cholesterol', 'High Pro...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...","Asian,Indian",https://www.food.com/recipe/biryani-39,weeknight,Dinner,dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,[['https://img.sndimg.com/food/image/upload/w_...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...",311.1,0.2,0.0,0.0,...,"Standard, Vegan, Vegetarian","['11⁄2 cupssugar', '1 tablespoonlemons, rind o...","['Low Protein', 'Medium Calorie', 'Low Fat', '...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...","North America, United States.",https://www.food.com/recipe/best-lemonade-40,summer,Drink,drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,[['https://img.sndimg.com/food/image/upload/w_...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...",536.1,24.0,3.8,0.0,...,"Standard, Vegetarian","['12 ouncesextra firm tofu, water-packed', '1 ...","['Medium Calorie', 'High Protein', 'High Sodiu...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",Global Food,https://www.food.com/recipe/carina%27s-tofu-ve...,weeknight,Dinner,dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,[['https://img.sndimg.com/food/image/upload/w_...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...",103.6,0.4,0.1,0.0,...,"Standard, Vegan, Vegetarian","['46 ouncesplain tomato juice', '4 cupscabbage...","['Low Carb', 'Low Protein', 'Low Fat', 'High S...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...","North America, United States",https://www.food.com/recipe/cabbage-soup-42,winter,Dinner,"dinner, lunch"


In [ ]:
# recipes_mealtype_nounknown = recipes_mealtype_nounknown.drop(columns='meal_type')
recipes_mealtype_nounknown.head()

,RecipeId,Name,Description,Images,RecipeCategory,Keywords,Calories,FatContent,SaturatedFatContent,CholesterolContent,...,cook_time,Diet_Types,ScrapedIngredients,Healthy_Type,RecipeIngredientQuantities,RecipeIngredientParts,RegionCountry,URL,occasion_updated,cleaned_meal_type
0,38,Low-Fat Berry Blue Frozen Dessert,Make and share this Low-Fat Berry Blue Frozen ...,[['https://img.sndimg.com/food/image/upload/w_...,Frozen Desserts,"Dessert, Low Protein, Low Cholesterol, Healthy...",170.9,2.5,1.3,8.0,...,more than 12 Hours,"Standard, Vegetarian","['4 cupsblueberries, fresh or frozen', '1⁄4 cu...","['Low Protein', 'Low Fat', 'Medium Fiber', 'Me...","4, 1/4, 1, 1","blueberries, granulated sugar, vanilla yogurt,...",Global Food,https://www.food.com/recipe/low-fat-berry-blue...,"summer, weeknight",dessert
1,39,Biryani,Make and share this Biryani .,[['https://img.sndimg.com/food/image/upload/w_...,Chicken Breast,"Chicken Thigh & Leg, Chicken, Poultry, Meat, A...",1110.7,58.8,16.6,372.8,...,less than 6 Hours,Standard,"['1 tablespoonsaffron', '4 teaspoonsmilk, warm...","['High Calorie', 'High Cholesterol', 'High Pro...","1, 4, 2, 2, 8, 1/4, 8, 1/2, 1, 1, 1/4, 1/4, 1/...","saffron, milk, hot green chili peppers, onions...","Asian,Indian",https://www.food.com/recipe/biryani-39,weeknight,dinner
2,40,Best Lemonade,This is from one of my first Good House Keepi...,[['https://img.sndimg.com/food/image/upload/w_...,Beverages,"Low Protein, Low Cholesterol, Healthy, Summer,...",311.1,0.2,0.0,0.0,...,less than 60 Mins,"Standard, Vegan, Vegetarian","['11⁄2 cupssugar', '1 tablespoonlemons, rind o...","['Low Protein', 'Medium Calorie', 'Low Fat', '...","1 1/2, 1, NA, 1 1/2, NA, 3/4","sugar, lemons, rind of, lemon, zest of, fresh ...","North America, United States.",https://www.food.com/recipe/best-lemonade-40,summer,drink
3,41,Carina's Tofu-Vegetable Kebabs,This dish is best prepared a day in advance to...,[['https://img.sndimg.com/food/image/upload/w_...,Soy/Tofu,"Beans, Vegetable, Low Cholesterol, Weeknight, ...",536.1,24.0,3.8,0.0,...,more than 12 Hours,"Standard, Vegetarian","['12 ouncesextra firm tofu, water-packed', '1 ...","['Medium Calorie', 'High Protein', 'High Sodiu...","12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1/2, 1/4, 4","extra firm tofu, eggplant, zucchini, mushrooms...",Global Food,https://www.food.com/recipe/carina%27s-tofu-ve...,weeknight,dinner
4,42,Cabbage Soup,Make and share this Cabbage Soup .,[['https://img.sndimg.com/food/image/upload/w_...,Vegetable,"Low Protein, Vegan, Low Cholesterol, Healthy, ...",103.6,0.4,0.1,0.0,...,less than 60 Mins,"Standard, Vegan, Vegetarian","['46 ouncesplain tomato juice', '4 cupscabbage...","['Low Carb', 'Low Protein', 'Low Fat', 'High S...","46, 4, 1, 2, 1","plain tomato juice, cabbage, onion, carrots, c...","North America, United States",https://www.food.com/recipe/cabbage-soup-42,winter,"dinner, lunch"


In [ ]:
recipes_mealtype_nounknown.rename(columns={'cleaned_meal_type': 'meal_type', 'occasion_updated': 'occasion'}, inplace=True)

In [ ]:
recipes_mealtype_nounknown.to_csv('/content/drive/MyDrive/Colab Notebooks/food_dataset/recipes_mealtype_nounknown.csv', index=False)